In [29]:
from housing_pricer.scraping.data_manager import DataManager
from _booli_processing_utils import *
import pandas as pd
from tqdm import tqdm
from itertools import islice
from joblib import Parallel, delayed

from bs4 import BeautifulSoup, Tag
import sys

import plotly.express as px
DATA_STORAGE = "../../scraping/booli_final_prices/data_storage"

# n_listings = 0
# for listings_html_content in tqdm(DataManager(DATA_STORAGE).load_data(), desc="Processing html content..."):
#     n_listings += 1
# n_listings

# LÄGENHET
# KUNGSHOLMEN
# ANTAL RUM
# AVGIFT
# DATUM SÅLD/BORTTAGEN
# VÅNING
# LGHNMR
# ENERGIKLASS
# DAGAR TILL SALU
# BOSTADSRÄTTSFÖRENING (BRF): BRF INFO (NAMN, ÄKTA BRF Y/N, ANTAL BOSTÄDER, LÅN)

Processing html content...: 30000it [00:06, 4902.21it/s]


30000

# Stripping html response of superfluous data

In [38]:
class DataProcessingError(Exception):
    """Exception raised for errors in data processing."""
    def __init__(self, msg: str):
        self.msg = msg
        super().__init__(self.msg)

def extract_relevant_data_as_json(html_content: str) -> dict:
    """
    Process the HTML content and keep only essential data in JSON format.
    
    Args:
    html_content: The HTML content as a string.

    Returns:
    dict: A dictionary containing the filtered JSON data.

    Raises:
    DataProcessingError: If any step in the data processing fails.
    """
    try:
        parsed_html = BeautifulSoup(html_content, "lxml")
        parsed_relevant_section = parsed_html.find("script", {"id": "__NEXT_DATA__"})

        if isinstance(parsed_relevant_section, Tag) and parsed_relevant_section.string:
            data_json = json.loads(parsed_relevant_section.string)
            filtered_data_json = {key: data_json[key] for key in ['props', 'page', 'query'] if key in data_json}
            return filtered_data_json
        else:
            raise DataProcessingError("Relevant script tag with specified id not found.")
    
    except json.JSONDecodeError as exc:
        raise DataProcessingError("Failed to decode JSON.") from exc
    except (KeyError, AttributeError) as exc:
        raise DataProcessingError("Error accessing data.") from exc
    except Exception:
        raise


# data_json = extract_relevant_data_as_json(listings_html_content)




In [42]:
import pickle
import gzip

with gzip.open("data_temp.gz", "ab") as fhandle:
    for listings_html_content in tqdm(DataManager(DATA_STORAGE).load_data(), desc="Re-converting data"):
        data_json = extract_relevant_data_as_json(listings_html_content)
        pickle.dump(data_json, fhandle)


Re-converting data: 30000it [03:28, 143.96it/s]


# Processing data

In [43]:
# N_LISTINGS_TO_PROCESS = 1000
# data = list(islice(DataManager(DATA_STORAGE).load_data(), N_LISTINGS_TO_PROCESS))

# def _process_listing_html(listing_html_content):
#     try:
#         listing_info = extract_ad_info(listing_html_content)
#         return listing_info
#     except Exception as exc:
#         print(exc)
#         return None

# parallel = Parallel(n_jobs=-1)
# delayed_jobs = (delayed(_process_listing_html)(html) for html in data)
# processed_listings = [job for job in parallel(delayed_jobs) if job is not None]
# pd.DataFrame(processed_listings)